[View in Colaboratory](https://colab.research.google.com/github/raahatg21/Creating-Artistic-Images/blob/master/style_transfer.ipynb)

# Neural Style Transfer

The basic workflow is as follows:

---



**1. Applying the Model to the images**


*   Load a pretrained model.
*   Load the *target_image* and *style_reference_image* as variables, and *combination_image* as placeholder. Concatenate all three.
*   Pass the batch through the model.


---




**2. Defining Individual Losses**


*   Define the *content_loss*.
*   Define the *style_loss* using the *gram_matrix*.
*   Define the *total_variation_loss*.


---



**3. Defining the Final Loss**


*   Find the right layer(s) for *content_loss* and *style_loss*.
*   Claculate the *layer_features*, i.e. activations of specific layer(s) for initial and combination image, and use them to calculate the loss.
*   The final loss is a weighted sum of all three losses.


---



**4. Optimization**


* Define *Evaluator* object, which calculates the *loss* and *gradient* simultaneously (used to overcome the limitation of the optimization algorithm).
* Start with the target image.
* Pass it throught the optimizer, so at each step it becomes closer to the desired image. Display the image at each step.
* Display the final image.


---



**5. Auxiliary Functions**


* Function to load and process images.
* Function to deprocess images.







In [1]:
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time
import os

from keras.applications import vgg19
from keras.preprocessing.image import load_img, img_to_array
from keras import backend as K

Using TensorFlow backend.


In [2]:
# Link Google Drive with Colaboratory

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
#!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#!apt-get update -qq 2>&1 > /dev/null
#!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [4]:
!ls

adc.json  google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb  wget-log
drive	  sample_data


In [5]:
!ls drive

cats_and_dogs_small		 Perl_1.pdf
CG				 Perl_2.pdf
CG.rar				 Perl_3.pdf
Colab Notebooks			 Perl_4.pdf
Deep_Dream_1.ipynb		 Perl_5.pdf
Deep_Dream_2.ipynb		 Perl_6.pdf
Dropbox_Chollet.pdf		 Perl_7.pdf
Email Doc.odt			 RoboFiesta 3.0 breif.odt
Financial Aid - Coursera.odt	 RoboFiesta 3.0 detail.odt
glove.6B.100d.txt		 Robotics
glove.6B.200d.txt		 Robotics.rar
Google Colab Key .odt		 sample-images
Images for Deep Dream		 Style_Transfer_1.ipynb
IMDB_Dataset			 TensorBoard
IMG-20171220-WA0015.jpg		 Untitled
jena_climate_2009_2016.csv	 Untitled0.ipynb
Language_Model_1.ipynb		 Untitled1.ipynb
my_model.h5			 Untitled (2093b74d)
New Doc 2018-09-01 11.27.40.odt  Untitled2.ipynb
Papers (1).zip			 Untitled3.ipynb
Papers (1).zip (Unzipped Files)  Untitled4.ipynb


In [0]:
base_dir = 'drive/sample-images'

In [0]:
target_dir = os.path.join(base_dir, 'content')
style_dir = os.path.join(base_dir, 'style')
final_dir = os.path.join(base_dir, 'final')

Change the content and style images here.

---



In [0]:
target_name = 'supercar-dubai.jpg'
target_path = os.path.join(target_dir, target_name)

style_name = 'marcus-d-lone-wolf.jpg'
style_path = os.path.join(style_dir, style_name)

In [0]:
# Let's keep the height at 400 px and adjust the width accordingly

width, height = load_img(target_path).size  # width, height = load_img(target_path).size
img_h = 480
img_w = int(width * img_h/height)

In [0]:
# Auxiliary Function 1

def preprocess_image(path):
  img = load_img(path, target_size = (img_h, img_w))
  img = img_to_array(img)
  img = np.expand_dims(img, axis = 0)  # Adding zeroth axis for no. of samples
  img = vgg19.preprocess_input(img)
  return img

In [0]:
# Auxiliary Function 2

def deprocess_image(x):
  x[:, :, 0] += 103.939  # Reversing the effects of vgg19.preprocess_input(image)
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:, :, ::-1]  # Converting from BGR to RGB
  x = np.clip(x, 0, 255).astype('uint8')
  return x

In [0]:
# Defining Constants/Placeholders for the three images

target_img = K.constant(preprocess_image(target_path))
style_img = K.constant(preprocess_image(style_path))
combination_img = K.placeholder((1, img_h, img_w, 3))

input_tensor = K.concatenate([target_img, style_img, combination_img], axis = 0)

In [15]:
# Loading the Model and applying it

model = vgg19.VGG19(input_tensor = input_tensor, weights = 'imagenet', include_top = False)
print('Model Loaded')

80142336/80134624 [==============================] - 7s 0us/step
Model Loaded


In [0]:
# Defining Content Loss

def content_loss(base, combination):
  return K.sum(K.square(combination - base))

In [0]:
# Defining Style Loss

def gram_matrix(x):
  features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
  gram = K.dot(features, K.transpose(features))
  return gram

def style_loss(style, combination):
  S = gram_matrix(style)
  C = gram_matrix(combination)
  channels = 3
  size = img_h * img_w
  return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [0]:
# We also define a 'Total Variation Loss'. It encourages spatial continuity on the generated image, avoiding overly pixelated results.

def total_variation_loss(x):
  a = K.square(x[:, :img_h - 1, :img_w - 1, :] - x[:, 1:, :img_w - 1, :])
  b = K.square(x[:, :img_h - 1, :img_w - 1, :] - x[:, :img_h - 1, 1:, :])
  return K.sum(K.pow((a + b), 1.25))

In [0]:
# Dictionary corresponding layer name to layer output

layer_dict = dict([(layer.name, layer.output) for layer in model.layers])

In [0]:
content_layer = 'block5_conv2'  # The last layer
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

Change the weight of content image here.

---



In [0]:
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025  # Change here for variation in results 

In [22]:
# Adding the content loss

loss = K.variable(0.)

layer_features = layer_dict[content_layer]
target_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]

loss += content_weight * content_loss(target_features, combination_features)

In [0]:
# Adding the style loss

for layer in style_layers:
  layer_features = layer_dict[layer]
  style_features = layer_features[1, :, :, :]
  combination_features = layer_features[2, :, :, :]
  
  loss += (style_weight / len(style_layers)) * style_loss(style_features, combination_features)

In [0]:
# Adding the total variation loss

loss += total_variation_weight * total_variation_loss(combination_img)

In [0]:
grads = K. gradients(loss, combination_img)[0]

fetch_loss_and_grads = K.function([combination_img], [loss, grads])

In [0]:
class Evaluator(object):
  def __init__(self):
    self.loss_val = None
    self.grads_val = None
    
  def loss(self, x):
    assert self.loss_val is None
    x = x.reshape((1, img_h, img_w, 3))
    outs = fetch_loss_and_grads([x])
    loss_val = outs[0]
    grads_val = outs[1].flatten().astype('float64')
    self.loss_val = loss_val
    self.grads_val = grads_val
    return self.loss_val
  
  def grads(self, x):
    assert self.loss_val is not None
    grads_val = np.copy(self.grads_val)
    self.loss_val = None
    self.grads_val = None
    return grads_val

evaluator = Evaluator()

In [27]:
# Minimizing the loss and getting the results

iterations = 20

x = preprocess_image(target_path)  # We start with the target_img and make changes to it to get combination_img
x = x.flatten()  # We need to flatten the input to this optimizer

for i in range(iterations):
  print('Iteration: {}'.format(i))
  start_time = time.time()
  
  x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime = evaluator.grads, maxfun = 20)
  print('Current loss: {}'.format(min_val))
  
  if (i+1) % 4 == 0:
    img = x.copy().reshape((img_h, img_w, 3))
    img = deprocess_image(img)
    fname = str(target_name[:-4]) + '-' + str(style_name[:-4]) + "-iter" + str(i) + ".jpg"
    file = os.path.join(final_dir, fname)
    imsave(file, img)
    print('Image saved')
  
  end_time = time.time()
  print('Time elapsed: {}\n'.format(end_time - start_time))

Iteration: 0
Current loss: 1449898496.0
Time elapsed: 29.085963010787964

Iteration: 1
Current loss: 676878016.0
Time elapsed: 24.30152702331543

Iteration: 2
Current loss: 451362560.0
Time elapsed: 24.616976737976074

Iteration: 3
Current loss: 348556352.0
Image saved
Time elapsed: 28.35849952697754

Iteration: 4
Current loss: 288577888.0
Time elapsed: 25.15135908126831

Iteration: 5
Current loss: 251711072.0
Time elapsed: 24.95712184906006

Iteration: 6
Current loss: 227023488.0
Time elapsed: 24.957969903945923

Iteration: 7
Current loss: 206498512.0
Image saved
Time elapsed: 29.78984522819519

Iteration: 8
Current loss: 191971776.0
Time elapsed: 25.04977035522461

Iteration: 9
Current loss: 178208080.0
Time elapsed: 25.098763704299927

Iteration: 10
Current loss: 169081616.0
Time elapsed: 24.88083815574646

Iteration: 11
Current loss: 159934720.0
Image saved
Time elapsed: 28.54998469352722

Iteration: 12
Current loss: 151275872.0
Time elapsed: 25.241708040237427

Iteration: 13
Curre